In [1]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
import pandas as pd
import numpy as np
from function import str_to_list
import re

In [2]:
df = pd.read_csv('csv/new_data.csv')
size = df.shape[0]


Data Preparation

In [3]:
def text_process(text):
    text = re.sub("\[|\]|'|,"," ",text)
    return text
    
df['text_replace'] = df['text'].apply(text_process)
df

,post_type,text,text_replace
0,3,"['หา', 'สัญญา', 'หอ', 'มจพ', 'ย้าย', 'ปลาย', '...",หา สัญญา หอ มจพ ย้าย ปลาย ...
1,1,"['ขาย', 'ตั๋วหนัง', 'black', 'panther', 'โรง',...",ขาย ตั๋วหนัง black panther โรง ...
2,1,"['อนุญาต', 'ขาย', 'ขาย', 'ตู้', 'เย็น', 'Midea...",อนุญาต ขาย ขาย ตู้ เย็น Midea...
3,1,"['XX', 'คน', 'เช่า', 'XX', 'อนุญาต', 'ประกาศ',...",XX คน เช่า XX อนุญาต ประกาศ ...
4,1,"['ขาย', 'สัญญา', 'หอ', 'หมู่บ้าน', 'สิ', 'ริน'...",ขาย สัญญา หอ หมู่บ้าน สิ ริน ...
...,...,...,...
1172,3,"['ร้านเตี๋ยว', 'เหอะ', 'ร้าน', 'ชา', 'ชัก', 'เ...",ร้านเตี๋ยว เหอะ ร้าน ชา ชัก เ...
1173,0,"['สายชาร์จ', 'Apple', 'ลืม', 'ห้อง', 'สมุด', '...",สายชาร์จ Apple ลืม ห้อง สมุด ...
1174,1,"['ขาย', 'เข็มขัด', 'หัว', 'ค้า', 'ซื้อ', '120'...",ขาย เข็มขัด หัว ค้า ซื้อ 120 ...
1175,0,"['แมว', 'หาย', 'เจอ', 'รบกวน', 'แจ้ง', 'หาย', ...",แมว หาย เจอ รบกวน แจ้ง หาย ...


In [4]:
X = df['text_replace']
y = df["post_type"]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=0)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=50,decode_error="ignore")
X_train_vectorized = vectorizer.fit_transform(X_train)

Built model

In [23]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB(alpha=1.0, fit_prior=True)
# transform the list of text to tf-idf before passing it to the model
cls.fit(vectorizer.fit_transform(X_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(vectorizer.fit_transform(X_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7728813559322034
              precision    recall  f1-score   support

           0       0.91      0.62      0.73        65
           1       0.71      0.81      0.75        99
           3       0.78      0.82      0.80       131

    accuracy                           0.77       295
   macro avg       0.80      0.75      0.76       295
weighted avg       0.79      0.77      0.77       295



In [8]:
import pickle

filename = "./model/check_type.sav"
filenamevec = "./model/vectorizer.sav"

#save model
pickle.dump(cls,open(filename,"wb"))
pickle.dump(vectorizer,open(filenamevec,"wb"))
